In [35]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import io
import os
import time

In [36]:
data_url = 'shakespeare_train.txt'
with io.open(data_url, 'r', encoding='utf8') as f:
    text = f.read()
    
# The unique characters in the file
vocab = sorted(set(text))

#construct character dictionary 
#vocab_to_int = {c:i for i,c in enumerate(vocab)}
#int_to_vocab =  dict(enumerate(vocab))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Encode data, shape = [number of characters]
train_data = np.array([char2idx[c] for c in text], dtype=np.int32)


In [37]:
idx2char[27]

'O'

In [38]:
train_data

array([18, 49, 58, ..., 52,  2,  0])

In [39]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('{:4s}:{:3d},'.format(repr(char), char2idx[char]))
print('...\n')

{
'\n':  0,
' ' :  1,
'!' :  2,
'$' :  3,
'&' :  4,
"'" :  5,
',' :  6,
'-' :  7,
'.' :  8,
'3' :  9,
':' : 10,
';' : 11,
'?' : 12,
'A' : 13,
'B' : 14,
'C' : 15,
'D' : 16,
'E' : 17,
'F' : 18,
'G' : 19,
...



In [40]:
#show how the first 13 characters from the text are mapped to integers
print('{} ----  characters mapped to int ----> {}'. format(repr(text[:13]),train_data[:13]))

'First Citizen' ----  characters mapped to int ----> [18 49 58 59 60  1 15 49 60 49 66 45 54]


In [44]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(train_data)

for i in char_dataset.take(5):
    
    #print('PRINT index', int(i))
    print(idx2char[i.numpy()]) 

F
i
r
s
t


In [51]:
'''sequences = char_dataset.batch(seq_length+1, drop_remainder= True)
for item in sequences.take(5):
    print(idx2char[item.numpy()])
    '''

'sequences = char_dataset.batch(seq_length+1, drop_remainder= True)\nfor item in sequences.take(5):\n    print(idx2char[item.numpy()])\n    '

In [47]:
#The batch method lets us easily convert these individual characters to sequences of the desired size.

sequences = char_dataset.batch(seq_length+1, drop_remainder= True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [52]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [55]:
# print the first examples in input and target values

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [56]:
'''Each index of these vectors are processed as one time step. For the input at time step 0, the model receives
the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing 
but the RNN considers the previous step context in addition to the current input character.'''

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 49 ('i')
Step    1
  input: 49 ('i')
  expected output: 58 ('r')
Step    2
  input: 58 ('r')
  expected output: 59 ('s')
Step    3
  input: 59 ('s')
  expected output: 60 ('t')
Step    4
  input: 60 ('t')
  expected output: 1 (' ')


In [61]:
#shuffle the data and pack it into batches.

# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [62]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [64]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

RuntimeError: tf.placeholder() is not compatible with eager execution.